In [15]:
import pandas as pd

## Data Collection & Preprocessing

### Data Collection and Key Purpose
Our key dataset is Divvy Bike Share Service data, from https://divvy-tripdata.s3.amazonaws.com/index.html. We selected data in year 2019, which consists of 4 csv files,from Q1 to Q4.

(Key Purpose to be filled up after machine learning part)

In [16]:
# Define the file paths for each quarter's data
file_paths = {
    'Q1': 'data/Divvy_Trips_2019_Q1.csv',
    'Q2': 'data/Divvy_Trips_2019_Q2.csv',
    'Q3': 'data/Divvy_Trips_2019_Q3.csv',
    'Q4': 'data/Divvy_Trips_2019_Q4.csv'
}

Check if the 4 files share identical column names in order to concatenate all quarters.

In [17]:
# Display the column names of each file to see the differences
for key, file_path in file_paths.items():
    df = pd.read_csv(file_path, nrows=1)  # Load just the first row for speed
    print(f"\nColumn names in {key}:")
    print(df.columns.tolist())



Column names in Q1:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']

Column names in Q2:
['01 - Rental Details Rental ID', '01 - Rental Details Local Start Time', '01 - Rental Details Local End Time', '01 - Rental Details Bike ID', '01 - Rental Details Duration In Seconds Uncapped', '03 - Rental Start Station ID', '03 - Rental Start Station Name', '02 - Rental End Station ID', '02 - Rental End Station Name', 'User Type', 'Member Gender', '05 - Member Details Member Birthday Year']

Column names in Q3:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']

Column names in Q4:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthy

It is clear that from the previous output, Q2 has different column names. Thus, we need to convert them to the same naming convention.

In [19]:
# Define the correct column names for consistency
correct_columns = [
    'trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 
    'from_station_id', 'from_station_name', 'to_station_id', 
    'to_station_name', 'usertype', 'gender', 'birthyear'
]

# Load each CSV file, rename columns, and store in a list
dfs = []
for key, file_path in file_paths.items():
    df = pd.read_csv(file_path)
    # Rename columns for Q2 as they do not match
    if key == 'Q2':
        df.columns = correct_columns
    dfs.append(df)

    # Display column names after renaming for verification
    print(f"\nColumn names in {key} after renaming:")
    print(df.columns.tolist())

# Concatenate all quarters into a single DataFrame
df_all_quarters = pd.concat(dfs, ignore_index=True)


Column names in Q1 after renaming:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']

Column names in Q2 after renaming:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']

Column names in Q3 after renaming:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']

Column names in Q4 after renaming:
['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'usertype', 'gender', 'birthyear']


### Data Preprocessing

**Step 1**

Convert the format of data.

In [20]:
# Convert 'start_time' and 'end_time' to datetime for easier manipulation
df_all_quarters['start_time'] = pd.to_datetime(df_all_quarters['start_time'], errors='coerce')
df_all_quarters['end_time'] = pd.to_datetime(df_all_quarters['end_time'], errors='coerce')

# Convert 'tripduration' to a numeric value (remove any commas and convert to float)
df_all_quarters['tripduration'] = pd.to_numeric(df_all_quarters['tripduration'].str.replace(',', ''), errors='coerce')

**Step 2**

Check missing values and replace them.

In [21]:
# Find the number of missing values in 'gender' and 'birthyear'
print("Missing values in 'start_time':", df_all_quarters['start_time'].isnull().sum())
print("Missing values in 'end_time':", df_all_quarters['end_time'].isnull().sum())
print("Missing values in 'tripduration':", df_all_quarters['tripduration'].isnull().sum())
print("Missing values in 'usertype':", df_all_quarters['usertype'].isnull().sum())
print("Missing values in 'gender':", df_all_quarters['gender'].isnull().sum())
print("Missing values in 'birthyear':", df_all_quarters['birthyear'].isnull().sum())

Missing values in 'start_time': 0
Missing values in 'end_time': 0
Missing values in 'tripduration': 0
Missing values in 'usertype': 0
Missing values in 'gender': 559206
Missing values in 'birthyear': 538751


In [22]:
# Replace missing values in 'gender' with "Unknown"
df_all_quarters['gender'].fillna('Unknown', inplace=True)

# Convert 'birthyear' to integers, filling missing values with a placeholder (e.g., 0 for unknown)
df_all_quarters['birthyear'] = df_all_quarters['birthyear'].fillna(0).astype(int)

# Check missing values in 'gender' after replacement
print("\nMissing values in 'gender' after replacement:", df_all_quarters['gender'].isnull().sum())

# Check missing values in 'birthyear' after replacement
print("\nMissing values in 'birthyear' after replacement:", df_all_quarters['birthyear'].isnull().sum())


Missing values in 'gender' after replacement: 0

Missing values in 'birthyear' after replacement: 0


**Step 3**

Check Duplicates.

In [23]:
# Check the number of duplicates
print("\nNumber of duplicate rows before removal:", df_all_quarters.duplicated().sum())

# Remove duplicates
df_all_quarters.drop_duplicates(inplace=True)

# Check the number of duplicates after removal
print("Number of duplicate rows after removal:", df_all_quarters.duplicated().sum())


Number of duplicate rows before removal: 0
Number of duplicate rows after removal: 0


**Step 4**

Feature Engineering

In [24]:
# Add a new column to categorize 'birthyear' into age groups
def categorize_birthyear(birthyear):
    if birthyear == 0:
        return "Unknown"
    elif birthyear < 1960:
        return "before 1960"
    elif 1960 <= birthyear < 1970:
        return "1960-1969"
    elif 1970 <= birthyear < 1980:
        return "1970-1979"
    elif 1980 <= birthyear < 1990:
        return "1980-1989"
    elif 1990 <= birthyear < 2000:
        return "1990-1999"
    else:
        return "after 1999"

df_all_quarters['age_group'] = df_all_quarters['birthyear'].apply(categorize_birthyear)

# Add columns for 'start_month' and 'start_day_of_week'
df_all_quarters['start_month'] = df_all_quarters['start_time'].dt.strftime('%b')  # Jan, Feb, ..., Dec
df_all_quarters['start_day_of_week'] = df_all_quarters['start_time'].dt.strftime('%a')  # Mon, Tue, ..., Sun


In [25]:
# Display a few rows to verify the changes
print("\nSample of processed data:")
print(df_all_quarters[['gender', 'birthyear', 'age_group', 'start_month', 'start_day_of_week']].head())


Sample of processed data:
   gender  birthyear  age_group start_month start_day_of_week
0    Male       1989  1980-1989         Jan               Tue
1  Female       1990  1990-1999         Jan               Tue
2  Female       1994  1990-1999         Jan               Tue
3    Male       1993  1990-1999         Jan               Tue
4    Male       1994  1990-1999         Jan               Tue
